In [1]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
data_path = "simple-examples/data"

In [4]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def load_data():
    # get the data paths
    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary

train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()

[9970, 9971, 9972, 9974, 9975]
{'but': 29, 'eliminate': 2531, 'printing': 3405, 'orleans': 4755, 'collected': 6353, 'dispatched': 6858, 'he': 28, 'journalism': 6491, 'daughter': 5360, 'tobacco': 2514, 'eugene': 6891, 'guarantees': 2002, 'lot': 581, 'victory': 2829, 'regrets': 9328, 'nagging': 9902, 'unauthorized': 7203, 'n.y.': 2226, 'maynard': 3101, 'rockwell': 4992, 'vanguard': 7216, 'mood': 4153, 'levy': 4934, 'terrorist': 9488, 'vacuum': 9544, 'one': 54, 'level': 461, 'defended': 5093, 'dai-ichi': 8818, 'plug': 8309, 'achieved': 2708, 'estimating': 8059, 'environmental': 1084, 'rocket': 9370, 'intensely': 9895, 'complaint': 3459, '190.58-point': 8570, 'cultural': 4878, 'argued': 2118, 'chaos': 6803, 'noted': 664, 'anti-drug': 7850, 'lending': 1712, 'unification': 7783, 'conducting': 4245, 'influence': 1909, 'thought': 948, 'mania': 9135, 'flaw': 8954, 'sure': 1137, 'grades': 7454, 'ford': 477, 'ranged': 5509, 'mca': 1972, 'whose': 624, 'sour': 5836, 'processed': 6171, 'located': 34

In [10]:
f = open(train_path) 

NameError: name 'train_path' is not defined

In [ ]:
all = f.read()

In [5]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

In [6]:
num_steps = 30
batch_size = 20
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

hidden_size = 500
use_dropout=True

In [8]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())
checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)
num_epochs = 50
run_opt = 1

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 500)           5000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 10000)         5010000   
_________________________________________________________________
activation_2 (Activation)    (None, 30, 10000)         0         
Total params: 14,014,000
Trainable params: 14,014,000
Non-trainable params: 0
__________________________________________

In [9]:
if run_opt == 1:
    model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])
    # model.fit_generator(train_data_generator.generate(), 2000, num_epochs,
    #                     validation_data=valid_data_generator.generate(),
    #                     validation_steps=10)
    model.save(data_path + "final_model.hdf5")
elif run_opt == 2:
    model = load_model(data_path + "\model-40.hdf5")
    dummy_iters = 40
    example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Training data:")
    for i in range(dummy_iters):
        dummy = next(example_training_generator.generate())
        print(dummy)
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_training_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps-1, :])
        true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)
    # test data set
    dummy_iters = 40
    example_test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Test data:")
    for i in range(dummy_iters):
        dummy = next(example_test_generator.generate())
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_test_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps - 1, :])
        true_print_out += reversed_dictionary[test_data[num_steps + dummy_iters + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)


W0812 16:43:33.235420 139704538892160 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
1549/1549 [==============================] - 884s 571ms/step - loss: 6.9797 - categorical_accuracy: 0.0509 - val_loss: 6.5925 - val_categorical_accuracy: 0.0601

Epoch 00001: saving model to simple-examples/data/model-01.hdf5
Epoch 2/50
1549/1549 [==============================] - 912s 589ms/step - loss: 6.5189 - categorical_accuracy: 0.0838 - val_loss: 6.5195 - val_categorical_accuracy: 0.0748

Epoch 00002: saving model to simple-examples/data/model-02.hdf5
Epoch 3/50
1549/1549 [==============================] - 929s 600ms/step - loss: 6.3294 - categorical_accuracy: 0.1075 - val_loss: 6.4705 - val_categorical_accuracy: 0.0866

Epoch 00003: saving model to simple-examples/data/model-03.hdf5
Epoch 4/50
1549/1549 [==============================] - 888s 573ms/step - loss: 6.2116 - categorical_accuracy: 0.1180 - val_loss: 6.4716 - val_categorical_accuracy: 0.0862

Epoch 00004: saving model to simple-examples/data/model-04.hdf5
Epoch 5/50
1549/1549 [==============================

1549/1549 [==============================] - 892s 576ms/step - loss: 5.2059 - categorical_accuracy: 0.1755 - val_loss: 6.7128 - val_categorical_accuracy: 0.1000

Epoch 00035: saving model to simple-examples/data/model-35.hdf5
Epoch 36/50
1549/1549 [==============================] - 893s 576ms/step - loss: 5.1708 - categorical_accuracy: 0.1792 - val_loss: 6.7341 - val_categorical_accuracy: 0.0971

Epoch 00036: saving model to simple-examples/data/model-36.hdf5
Epoch 37/50
1549/1549 [==============================] - 872s 563ms/step - loss: 5.1753 - categorical_accuracy: 0.1787 - val_loss: 6.7704 - val_categorical_accuracy: 0.0950

Epoch 00037: saving model to simple-examples/data/model-37.hdf5
Epoch 38/50
1549/1549 [==============================] - 890s 575ms/step - loss: 5.1175 - categorical_accuracy: 0.1839 - val_loss: 6.7859 - val_categorical_accuracy: 0.0932

Epoch 00038: saving model to simple-examples/data/model-38.hdf5
Epoch 39/50
1549/1549 [==============================] - 893